In [8]:
def get_postgress_data(query, user, password, col_index = None):
    """get data from payability posgress. In develop"""
    txt_ = 'postgresql://{}:{}@database.payability.com:5432/payability_v2'.format(user, password)
    engine = sqlalchemy.create_engine(txt_)
    data = pd.read_sql_query(sql, engine, index_col = col_index, parse_dates=True)
    
    return(data)
    

In [9]:
def transform_for_NaN_values(data, cls):
    """Function change NaN values to 1. In develop"""
    df = data.copy()
    df[cls] = df[cls].apply(lambda cell: np.where(pd.isnull(cell), 1, 0))
    return(df)

In [25]:
def return_transformed_data(df, status = 2, days = 14, window_ = 21, min_periods_ = 20):
    """In develop"""
    d = df.copy()
    status_prop = d.groupby('mp_sup_key')['status'].max()
    status_ = status_prop[status_prop == status]
    ds = d[d.mp_sup_key.isin(status_.index)]
    
    aggregations = {'cust_complaints_intelectual_prop': ['mean','max'],
     'cust_complaints_policy_violation': ['mean','max'],
     'cust_complaints_prod_authenticity': ['mean','max'],
     'cust_complaints_prod_safety': ['mean','max'],
     'cancellation_rate': ['mean','max'],
     'late_shipment_rate': ['mean','max'],
     'order_defect_rate': ['mean','max'],
     'status' : ['max']}
    
    sups_list = ds.mp_sup_key.unique()
    df_list = []

    for sup in sups_list:
        d = ds.loc[ds.mp_sup_key == sup]
        d = d.rolling(window=window_, min_periods=min_periods_).agg(aggregations)
        d['mp_sup_key'] = sup
        df_list.append(d)
    
    
    ds = pd.concat(df_list, axis= 'rows').dropna()
    
    ds.columns = [' '.join(col).strip() for col in ds.columns.values]
    
    ds.rename(columns= {'status max': 'status'}, inplace = True)
    ds.reset_index(inplace=True)
    cls = list(ds.columns.values)

    days_max = pd.DataFrame(ds.groupby('mp_sup_key')['status'].idxmax().dropna())
    days_max.rename(columns= {'status': 'index_suspended_first'}, inplace=True)
    ds = pd.merge(ds, days_max, left_on = 'mp_sup_key', right_on= 'mp_sup_key')
    ds['index_'] = ds.index
    ds = ds.loc[ds.index_ <= ds.index_suspended_first]
    ds['lag_status'] = ds.groupby('mp_sup_key')['status'].shift(-days)
    
    ds['max_filter'] = ds.groupby('mp_sup_key')['status'].transform(max)
    ds = ds.loc[ds['lag_status'] == ds['max_filter']]
    
    cls.append('lag_status')
    ds = ds[cls]
    
    return ds
    

In [30]:
def return_transformed_data_good_sup(df, days = 14, window_ = 21, min_periods_ = 20, sample = 300):
   """Lag status does not work well"""
    d = df.copy()
    status_prop = d.groupby('mp_sup_key')['status'].max()
    status_ = status_prop[status_prop == 0]
    ds = d[d.mp_sup_key.isin(status_.index)]
    
    
    aggregations = {'cust_complaints_intelectual_prop': ['mean','max'],
     'cust_complaints_policy_violation': ['mean','max'],
     'cust_complaints_prod_authenticity': ['mean','max'],
     'cust_complaints_prod_safety': ['mean','max'],
     'cancellation_rate': ['mean','max'],
     'late_shipment_rate': ['mean','max'],
     'order_defect_rate': ['mean','max'],
     'status' : ['max']}
    
    sups_list = ds.mp_sup_key.unique()
    sups_list = np.random.choice(sups_list, sample)
    df_list = []

    for sup in sups_list:
        d = ds.loc[ds.mp_sup_key == sup]
        d = d.rolling(window=window_, min_periods=min_periods_).agg(aggregations)
        d['mp_sup_key'] = sup
        df_list.append(d)
    
    
    ds = pd.concat(df_list, axis= 'rows').dropna()
    
    ds.columns = [' '.join(col).strip() for col in ds.columns.values]
    
    ds.rename(columns= {'status max': 'status'}, inplace = True)
    ds.reset_index(inplace=True)
    cls = list(ds.columns.values)
    
    ds['lag_status'] = ds.groupby('mp_sup_key')['status'].shift(-days)
    
    df_list2 = []
    for sup in sups_list:
        d = ds.loc[ds.mp_sup_key == sup]
        if len(d) < days + 1:
            continue
        d = d.iloc[[-days]]
        d['mp_sup_key'] = sup
        df_list2.append(d)
    
    ds = pd.concat(df_list2)
    
    
    cls.append('lag_status')
    ds = ds[cls]
    
    return ds